In [1]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import linalg
from numpy import linalg as LA
from sklearn.metrics import r2_score
from numpy.linalg import inv



def percentage_error(L_0,L):
    print(np.linalg.norm(L - L_0)/np.linalg.norm(L_0))
def generate_ARMA_sample(ar,ma,T):
    #np.random.seed(12345)
    arparams = np.array(ar)
    maparams = np.array(ma)
    ar = np.r_[1, -arparams] # add zero-lag and negate
    ma = np.r_[1, maparams] # add zero-lag
    y = sm.tsa.arma_generate_sample(ar, ma, T,1)
    return y
def generate_matrix_test_data(p1,p2,T):
    coeff_1=[0.3,0.5,0.8]
    coeff_2=[-0.2,-0.4,-0.6]
    choice_1=np.random.choice(coeff_1,size=6)
    choice_2=np.random.choice(coeff_2,size=6)
    f=np.array([generate_ARMA_sample([choice_1[i],choice_2[i]],[],T,) for i in range(6)]).reshape([3,2,T])
    #print(f.shape)
    ran_matrix=np.random.randn(p1,p1)
    U, s, Vh = linalg.svd(ran_matrix)
    R=U[:,:3]
    P=U[:,3:7]
    #print(R.shape)
    ran_matrix=np.random.randn(p2,p2)
    U, s, Vh = linalg.svd(ran_matrix)
    C=U[:,:2]
    D=U[:,2:5]
    #print(Right.shape)
    ran_matrix=np.random.randn(4,3,T)
    y=np.einsum('ik,klt,jl->ijt',R,f,C)+np.einsum('ik,klt,jl->ijt',P,ran_matrix,D)+np.random.normal(size=[p1,p2,T])
    return y,R,C,f,P,D,ran_matrix

def generate_vector_test_data(N,T,prt=False): #return data matrix,loadings matrix, factor matrix
    #need to set np.random.seed(0) for this to work as expected!
    #construct beta factors
    v_1,v_2,v_3=2*np.random.randn(T),1.7*np.random.randn(T),1.3*np.random.randn(T)
    # construct alpha factors
    def ar(coeff,lag,noise_std):  #generate ar(lag) process
        f=[np.random.randn() for _ in range(lag)]  #AR series
        for i in range(T):
            noise=np.random.randn()
            f.append(coeff*f[i]+noise*noise_std)
        return np.array(f[:T])
    f3=ar(0.9,3,1)
    f1=ar(0.8,1,1.5)
    #plt.plot(f1)
    f=np.array([f1,f3,v_1,v_2,v_3])
    sort_index = np.argsort(-f.std(axis=1))
    f=f[sort_index]
    if prt:
        print('off_diagonal sum=',np.sum(np.abs(f@f.T/T))-np.trace(f@f.T/T))
    #calcualte loading matrix as singular vectors of a random matrix
    ran_matrix=np.random.randn(N,50)
    U, s, Vh = linalg.svd(ran_matrix)
    loading=U[:,:5]
    #print(loading.T@loading)
    #calculate factor terms
    factor=loading@f
    #print(factor.mean(axis=1))
    #print(factor.std())
    #construct observation
    X=factor+np.random.randn(N,T)*factor.std()*0.5
    #true loadings and factors that satisfies our identification constrains
    U, s, Vh = linalg.svd(factor)
    Gamma=U[:,:len(f)]
    F=np.dot(Gamma.T,factor)
    #print(s[:len(f)+1])
    #print(F@F.T/T)
    return X,Gamma,F,loading,f

def calculate_pca(data,r):
    U, s, Vh = linalg.svd(data)
    Gamma=U[:,:r]
    F=np.dot(Gamma.T,data)
    return Gamma,F
#calculate the trace statistics
def trace_stat(A,B):
    return np.trace(A@B.T@inv(B@B.T)@B@A.T)/np.trace(A@A.T)

#evaluate loading matrix estimate
def compare_loading_plot(Gamma,Gamma_star):
    for i in range(len(Gamma_star.T)):
        fix,ax=plt.subplots()
        #ax.plot(loading[:,i])
        if r2_score(Gamma[:,i], Gamma_star[:,i])<r2_score(Gamma[:,i], -Gamma_star[:,i]):
            Gamma_star[:,i]=-Gamma_star[:,i]
            #print(i)
        ax.plot(Gamma[:,i])
        ax.plot(Gamma_star[:,i])
        print("r2=",r2_score(Gamma[:,i], Gamma_star[:,i]))     
def compare_factor_plot(F,F_star):
    for i in range(len(F_star)):
        fix,ax=plt.subplots()
        #ax.plot(loading[:,i])
        if r2_score(F[i], F_star[i])<r2_score(F[i], -F_star[i]):
            F_star[i]=-F_star[i]
            #print(i)
        ax.plot(F[i])
        ax.plot(F_star[i])
        print("r2=",r2_score(F[i], F_star[i]))                                   
def log_multivariate_normal_density(X, means, covars, min_covar=1e-7):
    """Log probability for full covariance matrices. """
    
    solve_triangular = linalg.solve_triangular
    n_samples, n_dim = X.shape
    nmix = len(means)
    log_prob = np.empty((n_samples, nmix))
    for c, (mu, cv) in enumerate(zip(means, covars)):
        for i in range(100):
            try:
                cv_chol = linalg.cholesky(cv \
                                          + i*1e-7 * np.eye(n_dim),lower=True)
                break
            except:
                continue
        cv_log_det = 2 * np.sum(np.log(np.diagonal(cv_chol)))
        cv_sol = solve_triangular(cv_chol, (X - mu).T, lower=True).T
        log_prob[:, c] = -0.5 * (np.sum(cv_sol ** 2, axis=1)  + cv_log_det)

    return log_prob
def compare_factors(model,ll,true_factor):
    smoothed_state_estimates=model.stacked_factor
    pca_factor=model.pca_factor
    for i in range(model.n_factor):
        pl.figure(figsize=(16, 6))
        if r2_score(true_factor[i][model.lag-1:], smoothed_state_estimates[i])<r2_score(true_factor[i][model.lag-1:], -smoothed_state_estimates[i]):
                true_factor[i][model.lag-1:]=-true_factor[i][model.lag-1:]
        print('r2 for pca estimates=',r2_score(true_factor[i][2:],model.pca_factor[i,2:]), ',  r2 for em estimates=',r2_score(true_factor[i][2:], model.stacked_factor[i]))
        lines_true = pl.plot(true_factor[i][2:], linestyle='-', color='b')
        lines_pca = pl.plot(model.pca_factor[i,2:], linestyle='--', color='r')
        lines_mine = pl.plot(model.stacked_factor[i], linestyle='-.', color='g')
        
        pl.legend(
            (lines_true[0], lines_pca[0], lines_mine[0]),
            ('true', 'pca', 'mine')
        )
        pl.xlabel('time')
        pl.ylabel('state')

    # Draw log likelihood of observations as a function of EM iteration number.
    # Notice how it is increasing (this is guaranteed by the EM algorithm)
    pl.figure()
    pl.plot(ll)
    pl.xlabel('em iteration number')
    pl.ylabel('log likelihood')
    pl.show()    
    
def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'C')

def select_r(e_values):           
    if len(e_values)==1:
        return 1
    ratios=np.roll(e_values,-1)[:-1]/e_values[:-1]
    return np.argmin(ratios)+1